# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](./images/ERD-week4-exercise.png)

## Instruction

1. สร้าง `virtual environment`
2. ติดตั้ง `django` และ `psycopg2` libraries
3. สร้างโปรเจคใหม่ใหม่ชื่อ`myshop`
4. จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`
5. สร้าง database ชื่อ `shop` ใน Postgres DB
6. ทำการเพิ่ม code ด้านล่างนี้ในไฟล์ `shop/models.py`
7. เพิ่ม **'shop'** ใน `settings.py`
8. ทำการ `makemigrations` และ `migrate`


```python
from django.db import models

# Create your models here.
class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)
```


**จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ `shop.sql` ใน PgAdmin4**

In [77]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from shop.models import *
from datetime import date
from django.db.models import Q, F

### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.25 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, 
ORDER ID:23, DATE: 2024-05-01, 
ORDER ID:24, DATE: 2024-05-01, 
ORDER ID:25, DATE: 2024-05-02, 
ORDER ID:26, DATE: 2024-05-02, 
ORDER ID:27, DATE: 2024-05-02, 
ORDER ID:28, DATE: 2024-05-03, 
ORDER ID:29, DATE: 2024-05-03, 
ORDER ID:30, DATE: 2024-05-03, 
ORDER ID:31, DATE: 2024-05-04, 
```

In [7]:
allOrder = Order.objects.filter(order_date__month = 5)[:10]
for i in allOrder:
    print(f"ORDER ID: {i.id}, DATE: {i.order_date},")


ORDER ID: 22, DATE: 2024-05-01,
ORDER ID: 23, DATE: 2024-05-01,
ORDER ID: 24, DATE: 2024-05-01,
ORDER ID: 25, DATE: 2024-05-02,
ORDER ID: 26, DATE: 2024-05-02,
ORDER ID: 27, DATE: 2024-05-02,
ORDER ID: 28, DATE: 2024-05-03,
ORDER ID: 29, DATE: 2024-05-03,
ORDER ID: 30, DATE: 2024-05-03,
ORDER ID: 31, DATE: 2024-05-04,


1.2 query หาข้อมูล `Order` ที่มีการซื้อสินค้าชื่อ "Dog Bed" (ดูที่รายการ `OrderItem`) และแสดงผลดังตัวอย่าง (0.25 คะแนน)

```txt
# ตัวอย่างบางส่วน
ORDER ID: 8, ORDER_DATE: 2024-04-26, CUSTOMER: Suwichada Jarastham, REMARK: None
```

In [28]:




allOrderDog = Order.objects.filter(orderitem__product__name = "Dog Bed")
for i in allOrderDog:
    print(f"ORDER ID: {i.id}, ORDER_DATE: {i.order_date}, CUSTOMER: {i.customer.first_name} {i.customer.last_name}, REMARK: {i.remark}")


ORDER ID: 8, ORDER_DATE: 2024-04-26, CUSTOMER: Suwichada Jarastham, REMARK: None
ORDER ID: 16, ORDER_DATE: 2024-04-29, CUSTOMER: Trinnapon Yinggnam, REMARK: None
ORDER ID: 23, ORDER_DATE: 2024-05-01, CUSTOMER: Warit Pititat, REMARK: None
ORDER ID: 36, ORDER_DATE: 2024-05-05, CUSTOMER: Manit Senapan, REMARK: None
ORDER ID: 63, ORDER_DATE: 2024-05-14, CUSTOMER: Dejwit Tangjareonsakul, REMARK: None
ORDER ID: 79, ORDER_DATE: 2024-05-20, CUSTOMER: Techathorn Sriwanwilai, REMARK: None
ORDER ID: 151, ORDER_DATE: 2024-06-13, CUSTOMER: Thitirat Sukkesorn, REMARK: None


1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และอยู่ในหมวดหมู่ `Information Technology` และแสดงผลดังตัวอย่าง (0.25 คะแนน)

```txt
# ตัวอย่างบางส่วน 
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
```

In [39]:
allProductIt = Product.objects.filter(price__gt = 5000, categories__name = "Information Technology")
for i in allProductIt:
    print(f"PRODUCT ID: {i.id}, NAME: {i.name}, PRICE: {i.price}")

PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00


1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.25 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00
```

In [43]:
products = Product.objects.filter(price__lt = 200, price__gt = 100)
for i in products:
    print(f"PRODUCT ID: {i.id}, NAME: {i.name}, PRICE: {i.price}")

PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00


1.5 query หาข้อมูล `Product` ในหมวดหมู่ `Electronics` หรือ `Toys and Games` ที่มีราคามากกว่าหรือเท่ากับ 10,000 หรือมีจำนวนคงเหลือน้อยกว่าหรือเท่ากับ 10 และแสดงผลดังตัวอย่าง (0.25 คะแนน)

```txt
# ตัวอย่างบางส่วน 
PRODUCT ID: 2,	PRICE: 25999.00, REMAINING: 19, CATEGORIES: Information Technology, Electronics
PRODUCT ID: 3,	PRICE: 8900.00, REMAINING: 10, CATEGORIES: Information Technology, Electronics
PRODUCT ID: 6,	PRICE: 5000.00, REMAINING: 10, CATEGORIES: Information Technology, Electronics
PRODUCT ID: 7,	PRICE: 32000.00, REMAINING: 4, CATEGORIES: Information Technology, Electronics
PRODUCT ID: 8,	PRICE: 990.00, REMAINING: 0, CATEGORIES: Information Technology, Electronics
PRODUCT ID: 8,	PRICE: 990.00, REMAINING: 0, CATEGORIES: Information Technology, Electronics
```

In [111]:
productEiei = Product.objects.filter(Q(categories__name = "Electronics") | Q(categories__name = "Toys and Games") | Q(price__gte = 10000))

<QuerySet [<Product: Product object (1)>, <Product: Product object (2)>, <Product: Product object (2)>, <Product: Product object (3)>, <Product: Product object (4)>, <Product: Product object (5)>, <Product: Product object (5)>, <Product: Product object (6)>, <Product: Product object (7)>, <Product: Product object (7)>, <Product: Product object (8)>, <Product: Product object (9)>, <Product: Product object (10)>, <Product: Product object (11)>, <Product: Product object (12)>, <Product: Product object (13)>, <Product: Product object (14)>, <Product: Product object (15)>, <Product: Product object (16)>, <Product: Product object (17)>, '...(remaining elements truncated)...']>


1.6 query หาข้อมูล Product โดยมีเงื่อนไขดังนี้: (0.25 คะแนน)

- ชื่อสินค้ามีคำว่า `Smart` หรือ `Electric`
- รายละเอียดสินค้า ไม่มีคำว่า `compact` หรือ `portable`
- รายละเอียดสินค้ามีชื่อสินค้า

และแสดงผลดังตัวอย่าง
```txt
PRODUCT ID: 1, NAME: Smartphone, DESCRIPTION: A sleek and powerful smartphone with advanced features. 
PRODUCT ID: 10, NAME: Smartwatch, DESCRIPTION: A stylish smartwatch with health monitoring and notification features. 
PRODUCT ID: 16, NAME: Electric Kettle, DESCRIPTION: Fast-boiling electric kettle with auto shut-off feature. 
PRODUCT ID: 18, NAME: Electric Fan, DESCRIPTION: Oscillating electric fan for cooling rooms. 
```

In [93]:
product2 = Product.objects.filter((Q(name__icontains = "Smart") | Q(name__icontains = "Electric")), 
                                  (~Q(description__icontains = "compact") | ~Q(description__icontains = "portable")), 
                                  description__icontains = F('name'))
                                    
 
for i in product2:
    print(f"PRODUCT ID: {i.id}, NAME: {i.name}, DESCRIPTION: {i.description}")

PRODUCT ID: 1, NAME: Smartphone, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 10, NAME: Smartwatch, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 16, NAME: Electric Kettle, DESCRIPTION: Fast-boiling electric kettle with auto shut-off feature.
PRODUCT ID: 18, NAME: Electric Fan, DESCRIPTION: Oscillating electric fan for cooling rooms.


### 2. เพิ่ม ลบ แก้ไข สินค้า

2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.1 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information Technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [105]:
book_cat = ProductCategory.objects.get(name = "Books and Media")
it_cat = ProductCategory.objects.get(name = "Information Technology")
elec_cat = ProductCategory.objects.get(name = "Electronics")

newProd1 = Product(name = "Philosopher's Stone (1997)", remaining_amount = 20, description = "By J. K. Rowling.", price = 790)
newProd2 = Product(name = "Me Before You", remaining_amount = 40, description = "A romance novel written by Jojo", price = 390)
newProd3 = Product(name = "Notebook HP Pavilion Silver", remaining_amount = 10, description = "Display Screen. 16.0", price = 20000)

newProd1.save()
newProd2.save()
newProd3.save()

newProd1.categories.add(book_cat)
newProd2.categories.add(book_cat)

newProd3.categories.add(it_cat)
newProd3.categories.add(elec_cat)

2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.1 คะแนน)

In [106]:
newProd1.name = "Half-Blood Prince (2005)"
newProd1.save()

2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.1 คะแนน)

In [107]:
book_cat = ProductCategory.objects.get(name = "Books and Media")
book_cat.name = "Books"
book_cat.save()

2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.2 คะแนน)

In [109]:
Product.objects.filter(categories__name = "Books").delete()

(8, {'shop.Product_categories': 4, 'shop.Product': 4})